In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector
%load_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit

In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="prisjakt_tv_norway"
uname="root"
pwd="4156"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [3]:
base_url = "https://www.prisjakt.no"
url = "https://www.prisjakt.no/c/tv"
page_links = []
links = []
page_links.append(url)
for page in page_links:
    url = page
    i = requests.get(url).text
    soup=BeautifulSoup(i,'html.parser')
    pages = soup.find_all("li",{"class":"PaginationListItem-sc-11oisbu-1 bKXcXk"})
    for page in pages:
        link = page.find("a", {"data-test":"PaginationLink"})
        links.append(link)
        if link == None:
            links.remove(link)
    for l in links:
        pg = l.get('href')
        full_url = base_url+pg
        while full_url not in page_links:
            page_links.append(full_url)


print(f'Pages Found: {len(page_links)}')

Pages Found: 33


In [4]:
productlists = []
for link in page_links:
        k = requests.get(link).text
        soup=BeautifulSoup(k,'html.parser')
        productlist = soup.find_all("article",{"data-test":"ProductGridCard"})
        productlists.append(productlist)

In [5]:
items = [item for sublist in productlists for item in sublist]

In [6]:
productlinks = []
for item in items:
        link = item.find("a",{"data-test":"InternalLink"}).get('href')                 
        productlinks.append(base_url+link)

print(f'Number of Products Found: {len(productlinks)}')

Number of Products Found: 1447


In [7]:
df_prod_links = pd.DataFrame(productlinks, columns = ['Product Links'])
# Convert dataframe to sql table                                   
df_prod_links.to_sql('productlinks', engine, if_exists='replace', index=False)

1447

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_26452\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
class Prices:
    def __init__(self, link):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.link = link
    def get_prices(self):
            self.driver.get(self.link)
            try:
                #  click on cookie banner
                banner_click = driver.find_element(By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)")
                banner_click.click()
            except:
                banner_click = None
            try:
                #click to reveal more items
                reveal_click = driver.find_element(By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton")
                reveal_click.click()
            except:
                reveal_click = None
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
            offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
            price_list = []
            for offer in offers:
                offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
                price_list.append(offer_prices)

            price_list = [price for sublist in price_list for price in sublist]
            for i in range(0, len(price_list)):
                price_list[i] = price_list[i].text.replace('\n',"")

            for i in range(0, len(price_list)):
                price_list[i] = re.sub('[\xa0]', '',price_list[i])
                #price_list[i] = re.sub('[kr]', '', price_list[i])
                price_list[i] = re.sub("[^0-9]", "", price_list[i])
                #price_list[i]= price_list[i].replace(r'[()]',"")

            for i in range(0, len(price_list)):
                price_list[i] = int(price_list[i])
            no_offers = len(price_list)
            avg_price =  "{:.2f}".format(mean(price_list))

            return no_offers, avg_price

In [ ]:
price_details = Prices(link)
price_details.get_prices()[1]

In [ ]:
data = []


In [ ]:

for link in productlinks[:10]:
    from datetime import datetime
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    price_details = Prices(link)
    try:
        #  click on cookie banner
        banner_click = driver.find_element(By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)")
        banner_click.click()
    except:
        banner_click = None
    try:
        #click to reveal more items
        reveal_click = driver.find_element(By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton")
        reveal_click.click()
    except:
        reveal_click = None

    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
    price_list = []
    for offer in offers:
        offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
        price_list.append(offer_prices)

    price_list = [price for sublist in price_list for price in sublist]
    for i in range(0, len(price_list)):
        price_list[i] = price_list[i].text.replace('\n',"")

    for i in range(0, len(price_list)):
        price_list[i] = re.sub('[\xa0]', '',price_list[i])
            #price_list[i] = re.sub('[kr]', '', price_list[i])
        price_list[i] = re.sub("[^0-9]", "", price_list[i])
            #price_list[i]= price_list[i].replace(r'[()]',"")

    for i in range(0, len(price_list)):
        price_list[i] = int(price_list[i])
    no_offers = len(price_list)
    avg_price =  "{:.2f}".format(mean(price_list))


    try:
        category = "TV"
    except:
        category = None
    try:
        name = hun.find("div",{"data-test":"ProductTitle"}).text.replace('\n',"")
    except:
        name = None
    try:
        avg_price = price_details.get_prices()[1]
    except:
        avg_price = None
    try:
        offs = price_details.get_prices()[0]
    except:
        offs = None
    try:
        country = "NO"
    except:
        country = None
    try:
        currency = "NOK"
    except:
        currency = None
    try:
        rank = hun.find("span",{"class":"Text--bzqghn kBBoFI titlesmalltext StyledCaptionText--7mhtyj fxXtPM"}).text.replace('\n',"")
    except:
        rank = None
    try:
        rating = hun.find("span",{"class":"RateNumber-sc-14ktvqu-5 ifhFUL"}).text.replace('\n',"")
    except:
        rating = None
    try:
        reviews = hun.find("span",{"class":"Counter-sc-14ktvqu-0 bORxLl"}).text.replace('\n',"")
    except:
        reviews = None
    try:
        description = hun.find("p",{"class":"ProductInfo-sc-46oo5c-0 bRxlhS"}).text.replace('\n',"")
    except:
        description = None
    #try:
        #specifications = get_specs(prod_specifications)
    #except:
        #specifications = None

    try:
        scrape_link = link
        
    except:
        scrape_link = None
    
    try:
        now = datetime.now()
        date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    except:
        date_scraped = None


    tv = {"Country Code":country, "Category":category, "Name":name, "Avg Price": avg_price, "Currency":currency, "Nb of Offers":offs, "Rank":rank, "Rating":rating, "Reviews":reviews, "Description":description, \
        "scrape_link":scrape_link, "date_scraped":date_scraped}
    
    data.append(tv)

In [ ]:
df_prop = pd.DataFrame.from_dict(property_list)
df_prop.head()

In [ ]:
df = pd.DataFrame.from_dict(data)
df.tail()

In [ ]:
df.columns

In [ ]:
df = pd.concat([df, df_prop.reindex(df.index)], axis=1)
#gen_spec_df = df["Generalspecifications"].apply(pd.Series)
#df = pd.concat([df, gen_spec_df.reindex(df.index)], axis=1)
df_c = df.copy()
# Remove specifications column
#df.drop(['specifications'], axis=1, inplace=True)
df.shape

In [ ]:
#df2 = df.reset_index().rename(columns={'index':'id'})
df_long = df.set_index(['Country Code', 'Produsenter','Kategori', 'Name', 'Nb of Offers', 'Lanseringsår', 'Avg Price', 'Currency', 'scrape_link', 'date_scraped']).stack().reset_index()

In [ ]:
# Convert dataframe to sql table                                   
df_long.to_sql('data_table',  engine, if_exists='replace', index=False)

In [ ]:
df.to_csv("data.csv")

    prod_specifications = hun.find_all("div",{"class":"Column-sc-1jnk5ag-2 bIOAXd"})
    def get_specs(prod_specifications):
        nam = []
        val = []
        for spec in prod_specifications:
            for prop in spec.find_all("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"}):
                prop_n  = prop.get_text()
                nam.append(prop_n)
            for prop in spec.find_all("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"}):
                prop_v  = prop.get_text()
                val.append(prop_v)
        specs = {nam[i]: val[i] for i in range(len(nam))}
        return(specs)

In [ ]:
df_long.to_csv('df_long.csv')